In this tutorial you will learn how to transform OBJ files to precomputed format and host them on local server to be displayed by local neuroglancer instance.

In [4]:
import os
import navis

from pyroglancer.volumes import uploadmultiresmeshes, uploadsingleresmeshes

In [5]:
local_path = "working/directory/path"

## Transform obj to precomp

In [3]:
foldname = "data/"
out_path = "precomp_mesh/"

if not os.path.isdir(out_path):
    os.mkdir(out_path)

Here we create list of of OBJ meshes, arbitrary numerical mesh ids and mesh labels that we will take from file names.

In [13]:
meshes = []
meshes_id = []
meshes_nm = []
for e,f in enumerate(os.listdir(foldname)):
    nrnmesh = navis.MeshNeuron(os.path.join(foldname, f))
    nrnmesh.segid = 100 + e # we add some arbitrary id to our meshes
    nrnmesh.vertices = nrnmesh.vertices*1000 # scale vertices to proper unit if necessary
    meshes.append(nrnmesh)
    meshes_id.append(100 + e)
    meshes_nm.append(f.split(".")[0])

In [7]:
type_of_precomputed = "single"

if type_of_precomputed == "single":
    uploadsingleresmeshes(meshes, meshes_id, meshes_nm, local_path, 'neuronmeshes')
elif type_of_precomputed == "single":
    uploadmultiresmeshes(meshes, meshes_id, meshes_nm, local_path, 'neuronmeshes')
else:
    raise Exception("Unrecognised precomputed type")

## Create pyroglancer scene

In [9]:
from pyroglancer.localserver import startdataserver, closedataserver

In [10]:
startdataserver(directory=local_path)

Serving data from:  /Users/dominik/projects/registration/precomp
Serving directory at http://127.0.0.1:8000


In [11]:
from pyroglancer.ngviewer import openviewer, closeviewer
from pyroglancer.layers import create_nglayer, setlayerproperty
from pyroglancer.ngspaces import create_ngspace
from pyroglancer.createconfig import createconfig
from pyroglancer.layers import get_ngserver, add_hostedlayer

Here you specify config YAML for your scene.

In [12]:
layer_kws = {'ngspace': 'MANC',
            "configfileloc": "/path.to/your/local/config.yml"}

create_ngspace(layer_kws)

config file loc is at: /Users/dominik/.pyroglancer/config.yml
using custom location at:  /Users/dominik/.pyroglancer/config.yml
Neuroglancer viewer created at:  http://127.0.0.1:56779/v/ef15241f8fb7f31a3fff45d958ab1adf38728a17/
config file loc is at: None
using default location at:  /Users/dominik/.pyroglancer/config.yml
Dimensions are in : MANC
Layer created:  image
config file loc is at: None
using default location at:  /Users/dominik/.pyroglancer/config.yml
Using layout : xy-3d
config file loc is at: None
using default location at:  /Users/dominik/.pyroglancer/config.yml
Dimensions are in : MANC
Layer created:  segmentation
config file loc is at: None
using default location at:  /Users/dominik/.pyroglancer/config.yml
Using layout : xy-3d


In [14]:
layer_serverdir, layer_host = get_ngserver()

We add our locally hosted layer and call it `obj_meshes`.

In [15]:
tmpviewer = add_hostedlayer(layer_kws = {'type': 'volumes',
                                         'name': 'obj_meshes',
                                         'host': layer_host,
                                         'alpha': 0.5})


127.0.0.1 - - [28/Jul/2021 11:18:17] "GET /precomputed/neuronmeshes/mesh/info HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 11:18:17] "GET /precomputed/neuronmeshes/mesh/segment_properties/info HTTP/1.1" 200 -


In [ ]:
closeviewer()
closedataserver()